# The Functools Module

The `functools` module provides functions that act on other functions. 

## A little background review

In Python, functions are "first class objects", meaning that you can pass them around in code just like other data types.

In this example, we'll create a function whose job it is to call another function. 



In [ ]:
import random

def function_caller(other_function):
    """ Accepts a function, announces it will run the function, then run it """
    print(f"About to run the function '{other_function.__name__}'!\n")
    result = other_function()
    print(f"Ran '{other_function.__name__}' and the result was: {result}")

def get_random_number():
    return random.randint(0, 100)

function_caller(get_random_number)

# remember that when you pass functions around,
# you should pass a reference to the function, not call it!
# function_caller(get_random_number()) 



Notice that we pass a *reference* to `get_random_number`, not the result of it. If we did that, the function would fail. In this case, it fails with an `AttributeError` on the very first line, since an `int` does not have a `__name__` attribute.

One other concept that's going to be helpful in learning `functools` is decorators. Decorators are a nice shorthand way to write **functions that return other functions**. Let's say you want some function to only run on a certain day of the week, but you can't or don't want to check the time manually everywhere it's called. You could wrap each call in another function that checks whether it can run:

In [ ]:
import time

def only_run_on_thursday(other_function):
    """ Checks whether it's Thursday, and if so, calls its argument"""
    if time.localtime().tm_wday == 3: # range [0, 6], Monday is 0
        other_function()
    else: 
        print(f"Could not run {other_function.__name__}, it's not Thursday!")
    
def say_hello():
    print("Hello there!")

only_run_on_thursday(say_hello)

That's a little clunky, so Python introduced decorators to eliminate some of that wordiness. We can get exactly the same functionality as above with the following code:

In [ ]:
import time

def only_run_on_thursday(other_function):
    def wrapper():
        if time.localtime().tm_wday == 3: # range [0, 6], Monday is 0
            other_function()
        else: 
            print(f"Could not run {other_function.__name__}, it's not Thursday!")
    return wrapper
    
@only_run_on_thursday
def say_hello():
    print("Hello there!")

say_hello()

This is nice, because callers of `say_hello` don't have to worry about whether it's Thursday or not. They can just call the function, and the decorator will take care of doing the time check every time. 

There's a lot of technical detail in exactly *how* this happens under the surface, but that's beyond the scope of this presentation. 

## Common Use Cases

### functools.reduce

When is passing functions around useful? Sometimes when you want to invoke a function many times on a set of arguments!

For example, you might want to `reduce` a set of numbers to a single value. A common example for teaching this is to get the product of a list of numbers, but we'll do something a little more interesting.

Let's reduce a list using an algorithm that adds even numbers and subtracts odd numbers:

In [ ]:
import functools

def add_evens_subtract_odds(current_value, next):
    print(f"Current: {current_value} \t Next: {next}")
    if next % 2 == 0:
        return current_value + next
    return current_value - next

numbers = [6, 5, 2, 1]

result = functools.reduce(add_evens_subtract_odds, numbers)
print("Result:", result)

# reduce also accepts a starting value:
# functools.reduce(add_evens_subtract_odds, numbers, 10)

# it's a good idea to use a default, because passing an empty list
# to `reduce` results in an error:
# functools.reduce(add_evens_subtract_odds, [])

It's a little weird that Python doesn't provide this as a built-in. Such is life! 

# functools.partial

Sometimes you want to temporarily redefine a function with specific arguments. This can be handy if you want to use a generic function in a specific way. Calling `functools.partial` will return a new function with some of the arguments pre-defined.

Here's an academic example:

In [ ]:
add_ten = functools.partial(add_two_numbers, 10)
add_ten(1) # should print 11

The `partial` function returns a `partial` object. If you want, you can inspect it to see what's going on under the surface:

In [ ]:
print("Original function:\t", add_ten.func)
print("Fixed arguments:\t", add_ten.args)
print("Fixed keywords:\t\t", add_ten.keywords)

# note that these are attributes added by `partial`. If you look for them on the original function, they don't exist:
# print(add_two_numbers.func) # Attribute error!


Here's a more practial example, direct from the Python docs. It creates a function that parses a string into a base 2 number instead of the default base 10:

In [ ]:
basetwo = functools.partial(int, base=2)

print(int("101010")) # defaults to base 10
basetwo("10100")

## Caching values

`functools` offers a few methods to cache values. There are 5, but we'll just look at these two:

- The `@cache` decorator, which stores the return value of a function for a given set of arguments. This is useful for speeding up deeply recursive functions. 
- The `@lru_cache` decorator, which is the same as `cache`, but it puts a limit on how big the cache can grow. If the cache is about to get too big, it drops the **L**east **R**ecently **U**sed value and adds the new one.

Here's an example of how you might use `@cache`:

(Note: remember that decorators are just functions that return other functions!)

In [62]:
# Example from the docs:
@functools.cache
def factorial(n):
    return n * factorial(n-1) if n else 1

factorial(10)      # no previously cached result, makes 11 recursive calls
factorial(5)       # just looks up cached value result
factorial(12)      # makes two new recursive calls, the other 10 are cached


# But let's prove it! We'll add a counter that gets incremented every time our function is called.
@functools.cache
def monitored_factorial(n):
    global num_calls
    # print("Called monitored_factorial with argument", n)
    num_calls += 1
    return n * monitored_factorial(n-1) if n else 1

num_calls = 0
monitored_factorial(10)
print(num_calls) # Should print 11


num_calls = 0
monitored_factorial(5)
print(num_calls) # Should print 0

num_calls = 0
monitored_factorial(12)
print(num_calls) # Should print 2

11
0
2


`@lru_cache` is similar, but it restricts the total number of cached items that can be stored. `@cache` is actually the equivalent of using `lru_cache` with `maxsize=None`. 

LRU caching is a good optimization to make when the results of the cached function are likely to favor a few commonly-requested values. For example, if your program reports the weather based on a location, you might want to cache the results for the user's local zip code, as well as a few nearby areas. 


In [45]:
# NOTE: to use this cell, you have to install theh `requests` library
# and the 'toml' library (for pip, that's `pip install requests`)

# You will also have to create a free account with tomorrow.io and get an api key

import functools
import json
import requests
import pprint
import toml 

config = toml.load("secrets.toml")
BASE_URL = "https://api.tomorrow.io/v4/timelines"
API_KEY = config["API_KEY"]

@functools.lru_cache(maxsize=2)
def request_weather(latitude, longitude):
    querystring = {
    "location": f"{str(latitude)}, {str(longitude)}", 
    "fields":["temperature"],
    "units":"metric",
    "timesteps":"1d",
    "apikey":API_KEY}

    response = requests.request("GET", BASE_URL, params=querystring)
    return json.loads(response.text)

request_weather(42, 71) # Boston
print(request_weather.cache_info())

request_weather(90, 135) # North Pole
print(request_weather.cache_info())

# request_weather(0, 0) # Null Island
# print(request_weather.cache_info())

request_weather(42, 71) # Boston again
print(request_weather.cache_info())


CacheInfo(hits=0, misses=1, maxsize=2, currsize=1)
CacheInfo(hits=0, misses=2, maxsize=2, currsize=2)
CacheInfo(hits=1, misses=2, maxsize=2, currsize=2)


If you need to, you can also clear the cache completely:

In [48]:
request_weather.cache_clear() # empty cache to start

request_weather(42, 71) # Boston
request_weather(0, 0)
print(request_weather.cache_info()) # should print two misses and no hits
request_weather(42, 71) # Boston 
print(request_weather.cache_info()) # should print one hit and the two misses from before

request_weather.cache_clear()

request_weather(42, 71) # Boston 
print(request_weather.cache_info()) # we expect to see one miss because the cache is empty again.

CacheInfo(hits=0, misses=2, maxsize=2, currsize=2)
CacheInfo(hits=1, misses=2, maxsize=2, currsize=2)
CacheInfo(hits=0, misses=1, maxsize=2, currsize=1)


`lru_cache` caches values based on function inputs, and by default it ignores type:

In [51]:
request_weather.cache_clear()

request_weather(0, 0)
print(request_weather.cache_info())
request_weather(0.0, 0)
request_weather(0.0, 0.0)


CacheInfo(hits=2, misses=1, maxsize=2, currsize=1)